In [1]:
# Core imports
import heyoka as hk
import pykep as pk
import numpy as np
from numpy import linalg as la
import pandas as pd
import math
from numpy.lib.function_base import append
from scipy.integrate import odeint
# Sys imports
import time
# Plot imports
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as mtick
# Other imports
from UROP_const import *
from UROP_aux_func import *

In [14]:
# create heyoka variables
sigma, tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4 = hk.make_vars("sigma", "tau", "zeta1", "zeta2", "zeta3",
                                                                "eta1", "eta2", "eta3", "eta4")

#  ?

In [15]:
# Step 1: Define Initial Conditions (dimensional)
r0 = np.array([0.0, -5888.9727, -3400.0]) #km   (class 'numpy.ndarray')
v0 = np.array([10.691338, 0.0, 0.0])      #km/s
[a0, e0_norm, i0, RAAN0, omega0, theta0] = rv2elorb(r0, v0, GMe)

In [16]:
# Step 2: non-dimensionalise 
r0nd = r0 / a0 #[km/km] = [-] since the orbit is highly elliptical normalise with the initial semimajor axis
               # otherwise use r0nd = r0/la.norm(r0)
v0nd = v0 * math.sqrt(a0/GMe)    #[-]
t0   = 0                         #s
t0nd = t0 / math.sqrt((a0**3)/GMe) #[-]

# Non-dimensionalise the parameters
GMend = 1 # GMe * a0d / (a0d * GMe) 
Rend  = Re / a0
rlnd  = rl / a0
omega_lnd = omega_l * math.sqrt((a0**3)/GMe)
GMlnd = GMl / GMe

In [17]:
# Step 3: tranform the non-dimensional ICs (r0nd, v0nd) in DROMO elements 
h0 = np.cross(r0nd, v0nd)                      # 3-components vector
e0 = - r0nd/la.norm(r0nd) - np.cross(h0, v0nd) # 3-components vector
# sigma0 from page 7. and from page 14. (initial conditions)
sigma0 = theta0 

In [18]:
# Initial state (sigma; tau, zeta1, zeta2, zeta3, eta1, eta2, eta3, eta4)
tau_0   = t0nd
zeta1_0 = la.norm(e0)
zeta2_0 = 0
zeta3_0 = 1/la.norm(h0)
eta1_0  = math.sin(i0/2)*math.cos((RAAN0-omega0)/2)
eta2_0  = math.sin(i0/2)*math.sin((RAAN0-omega0)/2)
eta3_0  = math.cos(i0/2)*math.sin((RAAN0+omega0)/2)
eta4_0  = math.cos(i0/2)*math.cos((RAAN0+omega0)/2)
S0 = [tau_0, zeta1_0, zeta2_0, zeta3_0, eta1_0, eta2_0, eta3_0, eta4_0]

In [19]:
# Valore finale di sigma
sigma_fin = 60*2*np.pi    # rad
fin_day =  288.1276894125 # days
# numero di passi per orbita
# m = 1000
# N = math.floor((sigma_fin - sigma0)/(2*np.pi)) * m
delta_sigma = 1   
N = math.floor((sigma_fin-sigma0)/delta_sigma - 1)
sigma_span = np.linspace(sigma0, sigma_fin, N)  

In [20]:
# Auxiliary relationships
s = 1 + zeta1 * hk.cos(hk.time) + zeta2 * hk.sin(hk.time)

In [21]:
# Keplerian motion
apx = 0
apy = 0
apz = 0

# J2

# moon

# How do you call the function dromo2rv at every time step ...



# Perturbing accelerations
a_px = apx/(zeta3**4 * s**3)
a_py = apy/(zeta3**4 * s**3)
a_pz = apz/(zeta3**4 * s**3)

In [22]:
#EOMs

# in heyoka is sigma equivalent to hk.time ?
# dtaudsigma    = 1/((zeta3**3) * (s**2)) 
# dzeta1dsigma  = s * math.sin(sigma)   * a_px   + (zeta1 + (1+s)*math.cos(sigma)) * a_py 
# dzeta2dsigma  = - s * math.cos(sigma) * a_px   + (zeta2 + (1+s)*math.sin(sigma)) * a_py
# dzeta3dsigma  = - zeta3 * a_pz
# deta1dsigma   = 1/2 * a_pz * (eta4 * math.cos(sigma) - eta3 * math.sin(sigma))
# deta2dsigma   = 1/2 * a_pz * (eta3 * math.cos(sigma) + eta4 * math.sin(sigma)),
# deta3dsigma   = 1/2 * a_pz * (-eta2* math.cos(sigma) + eta1 * math.sin(sigma)),
# deta4dsigma   = 1/2 * a_pz * (-eta1* math.cos(sigma) - eta2 * math.sin(sigma))
                    
# should it be sigma or hk.time as sigma is the independent variable ?
dtaudsigma    = 1/((zeta3**3) * (s**2)) 
dzeta1dsigma  = s * hk.sin(sigma)   * a_px   + (zeta1 + (1+s)*hk.cos(sigma)) * a_py 
dzeta2dsigma  = - s * hk.cos(sigma) * a_px   + (zeta2 + (1+s)*hk.sin(sigma)) * a_py
dzeta3dsigma  = - zeta3 * a_pz
deta1dsigma   = 1/2 * a_pz * (eta4 * hk.cos(sigma) - eta3 * hk.sin(sigma))
deta2dsigma   = 1/2 * a_pz * (eta3 * hk.cos(sigma) + eta4 * hk.sin(sigma)),
deta3dsigma   = 1/2 * a_pz * (-eta2* hk.cos(sigma) + eta1 * hk.sin(sigma)),
deta4dsigma   = 1/2 * a_pz * (-eta1* hk.cos(sigma) - eta2 * hk.sin(sigma))
                    

In [13]:
print('Compiling the Taylor integrator ... (this is done only once)')
start_time = time.time()

ta = hk.taylor_adaptive(sys = [(tau,dtaudsigma),(zeta1,dzeta1dsigma),
                               (zeta2,dzeta2dsigma),(zeta3,dzeta3dsigma),
                               (eta1,deta1dsigma),(eta2,deta2dsigma),
                               (eta3,deta3dsigma),(eta4,deta4dsigma)],
                                state = S0,
                                time = sigma0,
                                tol = 1e-13,
                                compact_mode = True)
end_time = time.time()

print('Done, in');
print("--- %s seconds ---" % (end_time - start_time))

print("\nHeyoka Taylor integrator:\n", ta)

# Can you give me any insight as to what the error means?

Compiling the Taylor integrator ... (this is done only once)


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. heyoka.core._taylor_adaptive_dbl(sys: List[Tuple[heyoka.core.expression, heyoka.core.expression]], state: List[float], time: float = 0.0, pars: List[float] = [], tol: float = 0.0, high_accuracy: bool = False, compact_mode: bool = False, t_events: List[heyoka::detail::t_event_impl<double>] = [], nt_events: List[heyoka::detail::nt_event_impl<double>] = [])
    2. heyoka.core._taylor_adaptive_dbl(sys: List[heyoka.core.expression], state: List[float], time: float = 0.0, pars: List[float] = [], tol: float = 0.0, high_accuracy: bool = False, compact_mode: bool = False, t_events: List[heyoka::detail::t_event_impl<double>] = [], nt_events: List[heyoka::detail::nt_event_impl<double>] = [])

Invoked with: [(tau, (1.0000000000000000 / ((square(zeta3) * zeta3) * square(((1.0000000000000000 + (zeta1 * cos(t))) + (zeta2 * sin(t))))))), (zeta1, 0.0000000000000000), (zeta2, 0.0000000000000000)

In [ ]:
# Here we redefine the initial conditions since we may want to change them without recompiling the integrator
ta.time = t0
# Note the python syntax to assign directly the array elements. Here necessary
# as heyoka does not allow to change the memory location of the state
ta.state[:] =  S0

In [ ]:
start_time = time.time()
out = ta.propagate_grid(sigma_span)
end_time = time.time()

print('Done, in');
print("--- %s seconds ---" % (end_time - start_time))
St = out[4] 
size = np.shape(St)
print(f'the shape of the solution (cartesian form) is {size}')